In [ ]:
import pandas as pd
import json
import numpy as np
import math

def process_seawea_data(csv1_path, csv2_path, output_json1, output_json2):
    """
    Process CSV1 (all reservoirs) and CSV2 (critical reservoirs) for the SEA-WEA dashboard
    
    Args:
        csv1_path: Path to 13-normalize_column_type_fixed_add_SEAWEA_ID.csv
        csv2_path: Path to 16_ciritical_reservoir_with_polygon_20250812_add_SEAWEA_ID_add_INDEX.csv
        output_json1: Output path for processed all reservoirs JSON
        output_json2: Output path for processed critical reservoirs JSON
    """
    
    print("🔄 Processing CSV1 (All reservoirs data)...")
    
    # Load CSV1 - all reservoirs
    df1 = pd.read_csv(csv1_path)
    
    # Remove columns to the right of dynamic_id as specified
    if 'dynamic_id' in df1.columns:
        dynamic_id_index = df1.columns.get_loc('dynamic_id')
        df1 = df1.iloc[:, :dynamic_id_index + 1]
    
    # Filter for required statuses only: Operational, Planned, Under_construction
    valid_statuses = ['Operational', 'Planned', 'Under_construction']
    df1_filtered = df1[df1['status'].isin(valid_statuses)].copy()
    
    print(f"📊 CSV1 Original records: {len(df1)}")
    print(f"📊 CSV1 After status filtering: {len(df1_filtered)}")
    print(f"📊 Status distribution: {df1_filtered['status'].value_counts().to_dict()}")
    
    # Process CSV1
    json1_data = process_dataframe_to_json(df1_filtered, "all_reservoirs")
    
    print("🔄 Processing CSV2 (Critical reservoirs data)...")
    
    # Load CSV2 - critical reservoirs
    df2 = pd.read_csv(csv2_path)
    
    print(f"📊 CSV2 Records: {len(df2)}")
    
    # Process CSV2
    json2_data = process_dataframe_to_json(df2, "critical_reservoirs")
    
    # Save JSON files
    save_json_safely(json1_data, output_json1)
    save_json_safely(json2_data, output_json2)
    
    print(f"✅ JSON1 saved to {output_json1}")
    print(f"✅ JSON2 saved to {output_json2}")
    
    # Return summary information
    return {
        'csv1_total': len(df1_filtered),
        'csv1_operational': len(df1_filtered[df1_filtered['status'] == 'Operational']),
        'csv1_future': len(df1_filtered[df1_filtered['status'].isin(['Planned', 'Under_construction'])]),
        'csv2_total': len(df2)
    }

def process_dataframe_to_json(df, dataset_type):
    """
    Convert DataFrame to JSON format suitable for the dashboard
    """
    # Replace invalid values with None
    df = df.replace(-999.9, None)
    df = df.replace(-99, None)
    df = df.where(pd.notnull(df), None)
    
    # Clean and validate coordinates
    df = df.dropna(subset=['longitude', 'latitude'])
    df = df[(df['longitude'] >= -180) & (df['longitude'] <= 180)]
    df = df[(df['latitude'] >= -90) & (df['latitude'] <= 90)]
    
    # Add classification flags for visualization
    df['is_critical_reservoir'] = False
    df['is_critical_hydropower'] = False
    
    # For reservoirs: critical if capacity >= 100 MCM
    if 'max_capacity_mcm' in df.columns:
        df['is_critical_reservoir'] = df['max_capacity_mcm'].fillna(0) >= 100
    if 'normal_capacity_mcm' in df.columns:
        df.loc[df['normal_capacity_mcm'].fillna(0) >= 100, 'is_critical_reservoir'] = True
    if 'min_capacity_mcm' in df.columns:
        df.loc[df['min_capacity_mcm'].fillna(0) >= 100, 'is_critical_reservoir'] = True
    
    # For hydropower: critical if power >= 30 MW
    if 'power_mw' in df.columns:
        df['is_critical_hydropower'] = df['power_mw'].fillna(0) >= 30
    
    # Add visualization categories
    df['capacity_category'] = df.apply(lambda row: get_capacity_category(row), axis=1)
    df['power_category'] = df.apply(lambda row: get_power_category(row), axis=1)
    
    # Create summary statistics
    summary = create_summary_stats(df, dataset_type)
    
    # Prepare final dataset structure
    processed_data = {
        'features': df.to_dict('records'),
        'summary': summary,
        'metadata': {
            'total_records': len(df),
            'dataset_type': dataset_type,
            'last_updated': pd.Timestamp.now().isoformat(),
            'filters': {
                'countries': sorted(df['country'].dropna().unique().tolist()) if 'country' in df.columns else [],
                'statuses': sorted(df['status'].dropna().unique().tolist()) if 'status' in df.columns else [],
                'main_uses': sorted(df['main_use'].dropna().unique().tolist()) if 'main_use' in df.columns else [],
                'basins': sorted(df['basin'].dropna().unique().tolist()) if 'basin' in df.columns else []
            }
        }
    }
    
    return processed_data

def get_capacity_category(row):
    """Determine capacity category for visualization"""
    # Check multiple capacity columns
    capacity_cols = ['max_capacity_mcm', 'normal_capacity_mcm', 'min_capacity_mcm']
    max_capacity = 0
    
    for col in capacity_cols:
        if col in row and pd.notnull(row[col]) and row[col] > 0:
            max_capacity = max(max_capacity, row[col])
    
    if max_capacity <= 0:
        return 'Unknown'
    elif max_capacity <= 10:
        return '0-10MCM'
    elif max_capacity <= 100:
        return '10-100MCM'
    elif max_capacity <= 1000:
        return '100-1000MCM'
    else:
        return '>1000MCM'

def get_power_category(row):
    """Determine power category for visualization"""
    power = row.get('power_mw', 0)
    
    if pd.isna(power) or power <= 0:
        return 'Unknown'
    elif power <= 30:
        return '0-30MW'
    elif power <= 100:
        return '30-100MW'
    elif power <= 1000:
        return '100-1000MW'
    else:
        return '>1000MW'

def create_summary_stats(df, dataset_type):
    """Create summary statistics for the dataset"""
    summary = {
        'total_count': len(df),
        'dataset_type': dataset_type
    }
    
    # Add country statistics
    if 'country' in df.columns:
        summary['countries'] = df['country'].value_counts().to_dict()
    
    # Add status statistics
    if 'status' in df.columns:
        summary['statuses'] = df['status'].value_counts().to_dict()
        summary['current_count'] = len(df[df['status'] == 'Operational'])
        summary['future_count'] = len(df[df['status'].isin(['Planned', 'Under_construction'])])
    
    # Add main use statistics
    if 'main_use' in df.columns:
        summary['main_uses'] = df['main_use'].value_counts().to_dict()
    
    # Add capacity and power distributions
    summary['capacity_distribution'] = df['capacity_category'].value_counts().to_dict()
    summary['power_distribution'] = df['power_category'].value_counts().to_dict()
    
    # Count critical facilities
    summary['critical_reservoirs'] = int(df['is_critical_reservoir'].sum())
    summary['critical_hydropower'] = int(df['is_critical_hydropower'].sum())
    
    return summary

def save_json_safely(data, output_file):
    """Save data to JSON with proper NaN handling"""
    def clean_nans(obj):
        """Recursively convert all NaN, inf, -inf to None for JSON safety"""
        if isinstance(obj, float):
            if math.isnan(obj) or math.isinf(obj):
                return None
            return obj
        elif isinstance(obj, dict):
            return {k: clean_nans(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [clean_nans(v) for v in obj]
        else:
            return obj
    
    # Clean processed_data recursively
    safe_data = clean_nans(data)
    
    # Save with proper formatting
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(safe_data, f, indent=2, allow_nan=False, ensure_ascii=False)

if __name__ == "__main__":
    # Example usage
    summary = process_seawea_data(
        csv1_path='13-allData_addSEAWEA_ID.csv',
        # csv1_path='14-reservoirs_30MW_and_100MCM.csv',
        csv2_path='16_ciriticalReservoir_addSEAWEA_ID_addINDEX.csv',
        output_json1='../processed_all_reservoirs.json',
        output_json2='../processed_critical_reservoirs.json'
    )
 
    print("📊 Processing Summary:")
    print(f"  Total reservoirs (filtered): {summary['csv1_total']}")
    print(f"  Current (Operational): {summary['csv1_operational']}")
    print(f"  Future (Planned + Under construction): {summary['csv1_future']}")
    print(f"  Critical reservoirs: {summary['csv2_total']}")

🔄 Processing CSV1 (All reservoirs data)...
📊 CSV1 Original records: 369
📊 CSV1 After status filtering: 369
📊 Status distribution: {'Operational': 369}
🔄 Processing CSV2 (Critical reservoirs data)...
📊 CSV2 Records: 360
✅ JSON1 saved to ../processed_all_reservoirs.json
✅ JSON2 saved to ../processed_critical_reservoirs.json
📊 Processing Summary:
  Total reservoirs (filtered): 369
  Current (Operational): 369
  Future (Planned + Under construction): 0
  Critical reservoirs: 360


C:\Users\15330\AppData\Local\Temp\ipykernel_6468\2558357492.py:83: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['is_critical_reservoir'] = df['max_capacity_mcm'].fillna(0) >= 100
C:\Users\15330\AppData\Local\Temp\ipykernel_6468\2558357492.py:85: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[df['normal_capacity_mcm'].fillna(0) >= 100, 'is_critical_reservoir'] = True
C:\Users\15330\AppData\Local\Temp\ipykernel_6468\2558357492.py:87: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a futur

In [2]:
import pandas as pd
import json
import numpy as np
import math

def process_seawea_data(csv1_path, csv2_path, output_json1, output_json2):
    """
    Process CSV1 (all reservoirs) and CSV2 (critical reservoirs) for the SEA-WEA dashboard
    
    Args:
        csv1_path: Path to 13-normalize_column_type_fixed_add_SEAWEA_ID.csv
        csv2_path: Path to 16_ciritical_reservoir_with_polygon_20250812_add_SEAWEA_ID_add_INDEX.csv
        output_json1: Output path for processed all reservoirs JSON
        output_json2: Output path for processed critical reservoirs JSON
    """
    
    print("🔄 Processing CSV1 (All reservoirs data)...")

    # Record which cols should be checked through and should be numerical
    num_cols = [
        "longitude", "latitude",
        "max_capacity_mcm", "normal_capacity_mcm", "min_capacity_mcm",
        "power_mw"
    ]
    
    # Load CSV1 - all reservoirs
    df1 = pd.read_csv(csv1_path)

    for col in num_cols:
        if col in df1.columns:
            # 去千分位逗号、去单位与非数字字符，再转数值
            s = df1[col].astype(str).str.strip().str.replace(",", "", regex=False)
            s = s.str.replace(r"[^0-9eE+\-\.]", "", regex=True)
            df1[col] = pd.to_numeric(s, errors="coerce")
    
    # Remove columns to the right of dynamic_id as specified
    if 'dynamic_id' in df1.columns:
        dynamic_id_index = df1.columns.get_loc('dynamic_id')
        df1 = df1.iloc[:, :dynamic_id_index + 1]
    
    # Filter for required statuses only: Operational, Planned, Under_construction
    valid_statuses = ['Operational', 'Planned', 'Under_construction']
    df1_filtered = df1[df1['status'].isin(valid_statuses)].copy()
    
    print(f"📊 CSV1 Original records: {len(df1)}")
    print(f"📊 CSV1 After status filtering: {len(df1_filtered)}")
    print(f"📊 Status distribution: {df1_filtered['status'].value_counts().to_dict()}")
    
    # Process CSV1
    json1_data = process_dataframe_to_json(df1_filtered, "all_reservoirs")
    
    print("🔄 Processing CSV2 (Critical reservoirs data)...")
    
    # Load CSV2 - critical reservoirs
    df2 = pd.read_csv(csv2_path)
    
    print(f"📊 CSV2 Records: {len(df2)}")
    
    

    for col in num_cols:
        if col in df2.columns:
            s = df2[col].astype(str).str.strip().str.replace(",", "", regex=False)
            s = s.str.replace(r"[^0-9eE+\-\.]", "", regex=True)
            df2[col] = pd.to_numeric(s, errors="coerce")

    
    # Process CSV2
    json2_data = process_dataframe_to_json(df2, "critical_reservoirs")
    
    # Save JSON files
    save_json_safely(json1_data, output_json1)
    save_json_safely(json2_data, output_json2)
    
    print(f"✅ JSON1 saved to {output_json1}")
    print(f"✅ JSON2 saved to {output_json2}")
    
    # Return summary information
    return {
        'csv1_total': len(df1_filtered),
        'csv1_operational': len(df1_filtered[df1_filtered['status'] == 'Operational']),
        'csv1_future': len(df1_filtered[df1_filtered['status'].isin(['Planned', 'Under_construction'])]),
        'csv2_total': len(df2)
    }

def process_dataframe_to_json(df, dataset_type):
    """
    Convert DataFrame to JSON format suitable for the dashboard
    """
    # Replace invalid values with None
    df = df.replace(-999.9, None)
    df = df.replace(-99, None)
    df = df.where(pd.notnull(df), None)
    
    # Clean and validate coordinates
    df = df.dropna(subset=['longitude', 'latitude'])
    df = df[(df['longitude'] >= -180) & (df['longitude'] <= 180)]
    df = df[(df['latitude'] >= -90) & (df['latitude'] <= 90)]
    
    # Add classification flags for visualization
    df['is_critical_reservoir'] = False
    df['is_critical_hydropower'] = False
    
    # For reservoirs: critical if capacity >= 100 MCM
    if 'max_capacity_mcm' in df.columns:
        df['is_critical_reservoir'] = df['max_capacity_mcm'].fillna(0) >= 100
    if 'normal_capacity_mcm' in df.columns:
        df.loc[df['normal_capacity_mcm'].fillna(0) >= 100, 'is_critical_reservoir'] = True
    if 'min_capacity_mcm' in df.columns:
        df.loc[df['min_capacity_mcm'].fillna(0) >= 100, 'is_critical_reservoir'] = True
    
    # For hydropower: critical if power >= 30 MW
    if 'power_mw' in df.columns:
        df['is_critical_hydropower'] = df['power_mw'].fillna(0) >= 30
    
    # Add visualization categories
    df['capacity_category'] = df.apply(lambda row: get_capacity_category(row), axis=1)
    df['power_category'] = df.apply(lambda row: get_power_category(row), axis=1)
    
    # Create summary statistics
    summary = create_summary_stats(df, dataset_type)
    
    # Prepare final dataset structure
    processed_data = {
        'features': df.to_dict('records'),
        'summary': summary,
        'metadata': {
            'total_records': len(df),
            'dataset_type': dataset_type,
            'last_updated': pd.Timestamp.now().isoformat(),
            'filters': {
                'countries': sorted(df['country'].dropna().unique().tolist()) if 'country' in df.columns else [],
                'statuses': sorted(df['status'].dropna().unique().tolist()) if 'status' in df.columns else [],
                'main_uses': sorted(df['main_use'].dropna().unique().tolist()) if 'main_use' in df.columns else [],
                'basins': sorted(df['basin'].dropna().unique().tolist()) if 'basin' in df.columns else []
            }
        }
    }
    
    return processed_data

def get_capacity_category(row):
    """Determine capacity category for visualization"""
    # Check multiple capacity columns
    capacity_cols = ['max_capacity_mcm', 'normal_capacity_mcm', 'min_capacity_mcm']
    max_capacity = 0
    
    for col in capacity_cols:
        if col in row and pd.notnull(row[col]) and row[col] > 0:
            max_capacity = max(max_capacity, row[col])
    
    if max_capacity <= 0:
        return 'Unknown'
    elif max_capacity <= 10:
        return '0-10MCM'
    elif max_capacity <= 100:
        return '10-100MCM'
    elif max_capacity <= 1000:
        return '100-1000MCM'
    else:
        return '>1000MCM'

def get_power_category(row):
    """Determine power category for visualization"""
    power = row.get('power_mw', 0)
    
    if pd.isna(power) or power <= 0:
        return 'Unknown'
    elif power <= 30:
        return '0-30MW'
    elif power <= 100:
        return '30-100MW'
    elif power <= 1000:
        return '100-1000MW'
    else:
        return '>1000MW'

def create_summary_stats(df, dataset_type):
    """Create summary statistics for the dataset"""
    summary = {
        'total_count': len(df),
        'dataset_type': dataset_type
    }
    
    # Add country statistics
    if 'country' in df.columns:
        summary['countries'] = df['country'].value_counts().to_dict()
    
    # Add status statistics
    if 'status' in df.columns:
        summary['statuses'] = df['status'].value_counts().to_dict()
        summary['current_count'] = len(df[df['status'] == 'Operational'])
        summary['future_count'] = len(df[df['status'].isin(['Planned', 'Under_construction'])])
    
    # Add main use statistics
    if 'main_use' in df.columns:
        summary['main_uses'] = df['main_use'].value_counts().to_dict()
    
    # Add capacity and power distributions
    summary['capacity_distribution'] = df['capacity_category'].value_counts().to_dict()
    summary['power_distribution'] = df['power_category'].value_counts().to_dict()
    
    # Count critical facilities
    summary['critical_reservoirs'] = int(df['is_critical_reservoir'].sum())
    summary['critical_hydropower'] = int(df['is_critical_hydropower'].sum())
    
    return summary

def save_json_safely(data, output_file):
    """Save data to JSON with proper NaN handling"""
    def clean_nans(obj):
        """Recursively convert all NaN, inf, -inf to None for JSON safety"""
        if isinstance(obj, float):
            if math.isnan(obj) or math.isinf(obj):
                return None
            return obj
        elif isinstance(obj, dict):
            return {k: clean_nans(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [clean_nans(v) for v in obj]
        else:
            return obj
    
    # Clean processed_data recursively
    safe_data = clean_nans(data)
    
    # Save with proper formatting
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(safe_data, f, indent=2, allow_nan=False, ensure_ascii=False)

if __name__ == "__main__":
    # Example usage
    summary = process_seawea_data(
        csv1_path='13-allData_addSEAWEA_ID-filtered.csv',
        # csv1_path='14-reservoirs_30MW_and_100MCM.csv',
        csv2_path='16_ciriticalReservoir_addSEAWEA_ID_addINDEX.csv',
        output_json1='../processed_all_reservoirs.json',
        output_json2='../processed_critical_reservoirs.json'
    )
 
    print("📊 Processing Summary:")
    print(f"  Total reservoirs (filtered): {summary['csv1_total']}")
    print(f"  Current (Operational): {summary['csv1_operational']}")
    print(f"  Future (Planned + Under construction): {summary['csv1_future']}")
    print(f"  Critical reservoirs: {summary['csv2_total']}")

🔄 Processing CSV1 (All reservoirs data)...
📊 CSV1 Original records: 2075
📊 CSV1 After status filtering: 1980
📊 Status distribution: {'Operational': 1685, 'Planned': 295}
🔄 Processing CSV2 (Critical reservoirs data)...
📊 CSV2 Records: 360
✅ JSON1 saved to ../processed_all_reservoirs.json
✅ JSON2 saved to ../processed_critical_reservoirs.json
📊 Processing Summary:
  Total reservoirs (filtered): 1980
  Current (Operational): 1685
  Future (Planned + Under construction): 295
  Critical reservoirs: 360
